# Vector Database Ingestion for Yoruba RAG System

## Objective
To transform preprocessed Yoruba textual data into vector
representations and store them in a vector database to support
efficient semantic retrieval for the Retrieval-Augmented Generation
(RAG) system.


In [ ]:
!pip install cohere --quiet
!pip install -U llama-index llama-index-llms-cohere --quiet
!pip install llama-index-llms-gemini --quiet
!pip install bert-score
!pip install -U weaviate-client --quiet
!pip install -U llama-index --quiet
%pip install -U llama-index-embeddings-huggingface --quieta
!pip install -U llama-index-vector-stores-weaviate --quiet
!pip install -U bitsandbytes --quiet
!pip install -U llama-index-llms-huggingface --quiet
%pip install llama-index-retrievers-bm25 --quiet
!pip install llama-index-embeddings-gemini


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
weaviate-client 4.19.2 requires protobuf<7.0.0,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
google-cloud-storage 2.19.0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-api-core 1.34.1 which is incompatible.
dataproc-spark-connect 0.7.5 requires google-api-core>=2.19, but you have google-api-core 1.34.1 which is incompatible.
bigframes 2.8.0 requires goog

In [6]:
pip install --upgrade protobuf>=4.21.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 6.33.2 which is incompatible.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.33.2 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.2 which is incompatible.
google-cloud-storage 2.19.0 requires google-api-core<3.0.0

In [ ]:
import os
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = "puifkdlvt8kgh7ga2rtuca.c0.us-west3.gcp.weaviate.cloud"
weaviate_api_key = "@piKey"



client =weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
)

print(client.is_ready())

True


In [8]:
import weaviate
from weaviate.classes.config import Configure
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType, ReferenceProperty
from weaviate.util import generate_uuid5
from weaviate.classes.query import QueryReference

In [20]:
# This is optional to empty your database
result = client.collections.delete("Yoruba_rag")
print(result)

None


In [10]:
from llama_index.core.embeddings import BaseEmbedding

from typing import Any, List

import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel
class AfriBERTaEmbedding(BaseEmbedding):
    _model: Any = None
    _tokenizer: Any = None
    _device: Any = None

    def __init__(
        self,
        model_name: str = "Davlan/afro-xlmr-mini",
        **kwargs: Any
    ) -> None:
        super().__init__(model_name=model_name, **kwargs)

        # CPU device
        self._device = torch.device("cpu")

        # 2. Load tokenizer (use_fast=False to avoid tokenizer.json issues)
        self._tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=False
        )

        # 3. Load model on CPU
        self._model = AutoModel.from_pretrained(model_name).to(self._device)
        self._model.eval()

    def _mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = (
            attention_mask.unsqueeze(-1)
            .expand(token_embeddings.size())
            .float()
        )
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
            input_mask_expanded.sum(1), min=1e-9
        )

    def _embed(self, texts: List[str]) -> List[List[float]]:
        """Core embedding logic"""
        inputs = self._tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512   # important for RAG
        ).to(self._device)

        with torch.no_grad():
            outputs = self._model(**inputs)

            # Pool
            embeddings = self._mean_pooling(
                outputs.last_hidden_state,
                inputs["attention_mask"]
            )

            # Normalize
            embeddings = F.normalize(embeddings, p=2, dim=1)

        return embeddings.tolist()

    # --- LlamaIndex required methods ---
    def _get_query_embedding(self, query: str) -> List[float]:
        return self._embed([query])[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        return self._embed([text])[0]

    def _get_text_embedding_batch(self, texts: List[str]) -> List[List[float]]:
        return self._embed(texts)

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)



In [ ]:
from huggingface_hub import login

# Paste your HuggingFace access token here
login("hf_your_token_here")

embedder = AfriBERTaEmbedding(use_gpu=True)

2026-01-04 17:16:08.045213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767546968.064601     243 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767546968.070429     243 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Some weights of XLMRobertaModel were not initialized from the model checkpoint at Davlan/afro-xlmr-mini and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
embedder

AfriBERTaEmbedding(model_name='Davlan/afro-xlmr-mini', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7e463dd17910>, num_workers=None, embeddings_cache=None)

In [1]:

import pandas as pd

import warnings
import os

!gdown --folder https://drive.google.com/drive/folders/1h7iKCPfQYCbR-cir2TJZE9XoMAlpSLtV?usp=sharing
warnings.filterwarnings('ignore')
path = "/kaggle/working/RAG document/"
files = os.listdir(path)
file_paths= [path + fo for fo in files if fo.endswith('.csv')]
file_paths

Retrieving folder contents
Retrieving folder 11CvPqZp2LGU7SUR_N-EGR2to6SnQ2v9N .ipynb_checkpoints
Processing file 1FBqoA1_SnAUTQrhq-TZcX9naCPqd29Lm bbc_yoruba_news.csv
Processing file 1IHbqXbi91jWTrZRH7srQHSNOnQk81iLP yankari_cleaned.csv
Processing file 1MMlkLQzCZfAaayzFBjAMGi92VYygUHXF yankari.ipynb
Processing file 1Qq0ZLb96ykNn7TmqPVEjpKnHGfrLfZLL yoruba_books_mistral.csv
Processing file 1L8YzloHLmvo-eZYVceYy6IBJ2UU7usvl yoruba_text_domains.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1FBqoA1_SnAUTQrhq-TZcX9naCPqd29Lm
To: /kaggle/working/RAG document/bbc_yoruba_news.csv
100%|███████████████████████████████████████| 3.56M/3.56M [00:00<00:00, 152MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IHbqXbi91jWTrZRH7srQHSNOnQk81iLP
To: /kaggle/working/RAG document/yankari_cleaned.csv
100%|██████████████████████████████████████| 86.0M/86.0M [00:01<00:00, 81.2MB/s]
Down

['/kaggle/working/RAG document/yoruba_books_mistral.csv',
 '/kaggle/working/RAG document/yankari_cleaned.csv',
 '/kaggle/working/RAG document/yoruba_text_domains.csv',
 '/kaggle/working/RAG document/bbc_yoruba_news.csv']

In [2]:
# Load all CSVs into a dictionary of DataFrames
dataframes = {path.split('/')[-1].replace('.csv', ''): pd.read_csv(path) for path in file_paths}

# Access them like this:
bbc_news_df = dataframes['bbc_yoruba_news']
domains_df = dataframes['yoruba_text_domains']
books_df = dataframes['yoruba_books_mistral']
yankari_df = dataframes['yankari_cleaned']
books_df['source'] = books_df['source'].str.replace('700267500-', '', regex=False)
bbc_news_df.drop(columns=['title', 'date'], inplace=True)
yankari_df.drop(columns=['type', 'source'], inplace=True)
yankari_df.rename(columns={'text': 'content'}, inplace=True)
bbc_news_df['domain']=bbc_news_df['domain'].str.lower()
yankari_df['domain']=yankari_df['domain'].str.lower()
books_df['domain']=books_df['domain'].str.lower()
domains_df['domain']=domains_df['domain'].str.lower()
yankari_df['domain'].replace({'fashion': 'social life'}, inplace=True)
yankari_df['domain'].replace({'sports': 'social life'}, inplace=True)
yankari_df['domain'].replace({'general': 'current affairs'}, inplace=True)
books_df['domain'].replace({'books': 'culture'}, inplace=True)
bbc_news_df['domain'].replace({'current_affairs': 'current affairs'}, inplace=True)
bbc_news_df['domain'].replace({'social_life': 'social life'}, inplace=True)
data=pd.concat([bbc_news_df, yankari_df, books_df, domains_df], ignore_index=True)
print(f"Shape of the combined data: {data.shape}")
data['content']=data['content'].str.replace('\n', ' ').str.replace('\r', ' ').str.replace('\t', ' ')
data['content']=data['content'].str.replace('  ', ' ').str.replace('  ', ' ').str.replace('  ', ' ')
data['content']=data['content'].str.strip()
data.drop_duplicates(subset=['content'], inplace=True,keep='first')
print(f"Shape of the combined data after removing duplicates: {data.shape}")
data.reset_index(drop=True, inplace=True)


Shape of the combined data: (181052, 5)
Shape of the combined data after removing duplicates: (179646, 5)


In [4]:
import re
import unicodedata

# 1. Noise Removal: HTML tags, URLs, boilerplate, formatting
def remove_noise(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', ' ', text)
    # Remove boilerplate (customize as needed)
    text = re.sub(r'(copyright|all rights reserved|page \d+)', ' ', text, flags=re.IGNORECASE)
    # Remove extra formatting
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 2. Text Normalization: lowercasing, punctuation, whitespace
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'([.,!?;:])', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 3. Yorùbá Unicode Normalization
def yoruba_unicode_normalize(text):
    return unicodedata.normalize('NFC', text)


# Full Preprocessing Pipeline
def preprocess_pipeline(text):
    text = remove_noise(text)
    text = normalize_text(text)
    text = yoruba_unicode_normalize(text)
    return text


data['content'] = data['content'].apply(preprocess_pipeline)

In [21]:
if not client.collections.exists("Yoruba_rag"):
    client.collections.create(
        name="Yoruba_rag",
        vectorizer_config=None,  # external embeddings
        properties=[
            Property(name="chunks_no", data_type=DataType.TEXT),
            Property(name="text", data_type=DataType.TEXT),
            Property(name="domain", data_type=DataType.TEXT),
            Property(name="source", data_type=DataType.TEXT),
            Property(name="url", data_type=DataType.TEXT),
            Property(name="chunking_strategy", data_type=DataType.TEXT),
            Property(name="original_id", data_type=DataType.INT),
        ]
    )


In [22]:
import numpy as np
import re
from uuid import uuid4
from tqdm import tqdm

from llama_index.core import StorageContext
from llama_index.core.schema import TextNode
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [23]:
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="Yoruba_rag",
    text_key="text"
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [24]:
# ============================================
# IMPORTS
# ============================================
import numpy as np
from uuid import uuid4
from tqdm import tqdm
import re



# ============================================
# DOMAIN CONFIG & SMART SPLITTER
# ============================================
DOMAIN_CONFIG = {
    "religion":     {"chunk_size": 550, "chunk_overlap": 70},
    "culture":      {"chunk_size": 500, "chunk_overlap": 60},
    "social_life":  {"chunk_size": 400, "chunk_overlap": 50},
    "current_affairs": {"chunk_size": 450, "chunk_overlap": 50},
    "entertainment": {"chunk_size": 350, "chunk_overlap": 40},
    "default":      {"chunk_size": 400, "chunk_overlap": 50}
}

def smart_split_text(text: str, domain: str) -> list:
    config = DOMAIN_CONFIG.get(domain, DOMAIN_CONFIG["default"])
    chunk_size, overlap = config["chunk_size"], config["chunk_overlap"]

    text = re.sub(r'\s+', ' ', text.strip())

    if domain in ["religion", "culture"]:
        sentences = re.split(r'(\n\n|\. |\? |! |Òwe:|Ifá sọ pé:|Báwo ni:|Ìdí ti:)', text)
    else:
        sentences = re.split(r'([.!?]+)', text)

    reconstructed = []
    for i in range(0, len(sentences), 2):
        chunk = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
        if chunk.strip():
            reconstructed.append(chunk.strip())

    chunks = []
    current = ""
    for sent in reconstructed:
        if len(current) + len(sent) <= chunk_size:
            current += " " + sent
        else:
            if current:
                chunks.append(current.strip())
            current = (current[-overlap:] + " " + sent) if len(current) > overlap else sent

    if current:
        chunks.append(current.strip())

    return chunks if chunks else [text[:chunk_size]]

In [26]:
ROW_BATCH_SIZE = 3000
EMBED_BATCH_SIZE = 128

BATCH_SIZE = 3000   # rows, not chunks


batches = np.array_split(data, max(1, len(data) // BATCH_SIZE))

print(f"Total rows to process: {len(data)}")
print(f"Processing in {len(batches)} batches...")


Total rows to process: 179646
Processing in 59 batches...


In [29]:
import os
import json
import time

CHECKPOINT_FILE = "ingestion_checkpoint.json"

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            return json.load(f)
    return {"last_batch": -1, "total_nodes": 0}

def save_checkpoint(batch_id, total_nodes):
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(
            {
                "last_batch": batch_id,
                "total_nodes": total_nodes,
                "timestamp": time.time()
            },
            f
        )


In [30]:
checkpoint = load_checkpoint()

In [31]:
ROW_BATCH_SIZE = 3000
EMBED_BATCH_SIZE = 128
LOG_EVERY = 1   # batches

checkpoint = load_checkpoint()
start_batch = checkpoint["last_batch"] + 1
total_docs = checkpoint["total_nodes"]

start_time = time.time()

for batch_id in range(start_batch, len(batches)):
    batch = batches[batch_id]

    all_chunks = []
    chunk_meta = []

    # ---- FAST ROW LOOP ----
    for row in batch.itertuples(index=True):
        idx = row.Index
        text = getattr(row, "content", None)

        if not text or not isinstance(text, str):
            continue

        domain = getattr(row, "domain", "default")
        domain_key = domain.lower().replace(" ", "_")
        if domain_key not in DOMAIN_CONFIG:
            domain_key = "default"

        base_meta = {
            "domain": domain,
            "source": getattr(row, "source", None),
            "url": getattr(row, "url", None),
            "original_id": int(idx),
            "chunking_strategy": "domain_aware"
        }

        chunks = smart_split_text(text, domain_key)

        for chunk_no, chunk in enumerate(chunks, start=1):
            all_chunks.append(chunk)
            chunk_meta.append({
                **base_meta,
                "chunk_no": chunk_no,
                "node_id": str(uuid4())
            })

    # ---- EMBEDDING (BATCHED) ----
    nodes = []

    for i in range(0, len(all_chunks), EMBED_BATCH_SIZE):
        texts = all_chunks[i:i + EMBED_BATCH_SIZE]
        metas = chunk_meta[i:i + EMBED_BATCH_SIZE]

        embeddings = embedder.get_text_embedding_batch(texts)

        for emb, txt, meta in zip(embeddings, texts, metas):
            nodes.append(
                TextNode(
                    id_=meta["node_id"],
                    text=txt,
                    embedding=emb,
                    metadata={k: v for k, v in meta.items() if k != "node_id"}
                )
            )

    # ---- WEAVIATE WRITE ----
    if nodes:
        vector_store.add(nodes)
        total_docs += len(nodes)

    # ---- CHECKPOINT ----
    save_checkpoint(batch_id, total_docs)

    # ---- LOGGING ----
    elapsed = time.time() - start_time
    batches_done = batch_id + 1
    batches_left = len(batches) - batches_done
    avg_time = elapsed / batches_done
    eta = avg_time * batches_left

    print(
        f"✅ Batch {batch_id + 1}/{len(batches)} | "
        f"Nodes: {total_docs} | "
        f"Elapsed: {elapsed/60:.1f} min | "
        f"ETA: {eta/60:.1f} min"
    )

print(f"\n🎉 INGESTION COMPLETE — {total_docs} nodes indexed")


✅ Batch 1/59 | Nodes: 6756 | Elapsed: 10.6 min | ETA: 617.4 min
✅ Batch 2/59 | Nodes: 9801 | Elapsed: 17.7 min | ETA: 505.5 min
✅ Batch 3/59 | Nodes: 12846 | Elapsed: 25.7 min | ETA: 479.8 min
✅ Batch 4/59 | Nodes: 15891 | Elapsed: 31.1 min | ETA: 427.1 min
✅ Batch 5/59 | Nodes: 18936 | Elapsed: 35.5 min | ETA: 383.0 min
✅ Batch 6/59 | Nodes: 21981 | Elapsed: 43.3 min | ETA: 382.6 min
✅ Batch 7/59 | Nodes: 25026 | Elapsed: 51.3 min | ETA: 380.7 min
✅ Batch 8/59 | Nodes: 28071 | Elapsed: 59.4 min | ETA: 378.5 min
✅ Batch 9/59 | Nodes: 31116 | Elapsed: 67.3 min | ETA: 373.8 min
✅ Batch 10/59 | Nodes: 34161 | Elapsed: 74.9 min | ETA: 366.9 min
✅ Batch 11/59 | Nodes: 37206 | Elapsed: 80.0 min | ETA: 349.1 min
✅ Batch 12/59 | Nodes: 40251 | Elapsed: 83.1 min | ETA: 325.4 min
✅ Batch 13/59 | Nodes: 43296 | Elapsed: 85.9 min | ETA: 304.1 min
✅ Batch 14/59 | Nodes: 46341 | Elapsed: 88.9 min | ETA: 285.9 min
✅ Batch 15/59 | Nodes: 49386 | Elapsed: 91.7 min | ETA: 268.9 min
✅ Batch 16/59 | Nodes

In [4]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
import numpy as np
from tqdm import tqdm

# ============================================
# CONFIGURATION
# ============================================
BATCH_SIZE = 1000     # Adjust based on RAM (1000–5000 recommended)
CHUNK_SIZE = 350
CHUNK_OVERLAP = 50



# Initialize splitter
splitter = SentenceSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

# ============================================
# 1. SPLIT DATAFRAME INTO BATCHES
# ============================================
batches = np.array_split(data, max(1, len(data) // BATCH_SIZE))

all_nodes = []
total_docs = 0

# ============================================
# 2. PROCESS EACH BATCH
# ============================================
print(f"Processing {len(batches)} batches...")

for batch_id, batch in enumerate(tqdm(batches, desc="Batch Processing")):

    # Convert batch rows → Documents (vectorized approach)
    documents = []
    for idx, row in batch.iterrows():
        content = row.get("content")
        if not content:
            continue

        metadata = {
            'domain': row.get('domain'),
            'source': row.get('source'),
            'url': row.get('url'),
        }

        # Use original index as doc ID
        documents.append(
            Document(
                text=content,
                metadata=metadata,
                id_=str(idx)
            )
        )

    # Update total docs count
    total_docs += len(documents)

    if not documents:
        continue

    # ============================================
    # 3. SPLIT INTO NODES WITH SENTENCE SPLITTER
    # ============================================
    batch_nodes = splitter.get_nodes_from_documents(documents)

    # ============================================
    # 4. FIX NODE IDS (docID-chunk-XXX)
    # ============================================
    doc_chunk_map = {}  # keeps chunk number for each doc

    for node in batch_nodes:
        doc_base_id = node.ref_doc_id

        # auto-increment chunk_no for each doc
        doc_chunk_map.setdefault(doc_base_id, 0)
        doc_chunk_map[doc_base_id] += 1
        chunk_no = doc_chunk_map[doc_base_id]

        # Create nice ID
        new_node_id = f"{doc_base_id}-chunk-{chunk_no:03d}"
        node.id_ = new_node_id
        node.metadata["chunks_no"] = chunk_no

    all_nodes.extend(batch_nodes)

# ============================================
# RESULTS
# ============================================

print("====================================")
print(f"Total Documents Processed: {total_docs}")
print(f"Total Nodes (Chunks): {len(all_nodes)}")
if all_nodes:
    print(f"Example Node ID: {all_nodes[0].id_}")
print("====================================")


Processing 179 batches...


Batch Processing: 100%|██████████| 179/179 [04:00<00:00,  1.34s/it]

Total Documents Processed: 179646
Total Nodes (Chunks): 402101
Example Node ID: 0-chunk-001


In [21]:
all_nodes[0].metadata.get('domain')

'entertainment'

In [24]:
len(all_nodes)

402101

In [ ]:

# --- Create collection if it doesn't exist ---
if not client.collections.exists("Yoruba_rag"):
    collection = client.collections.create(
        name="Yoruba_rag",
        vectorizer_config=None,  # we provide our own embeddings
        properties=[
            Property(name="chunks_no", data_type=DataType.TEXT),
            Property(name="text", data_type=DataType.TEXT),
            Property(name="domain", data_type=DataType.TEXT),
            Property(name="source", data_type=DataType.TEXT),
            Property(name="url", data_type=DataType.TEXT),
        ]
    )

# --- Insert documents with embeddings ---
yoruba_collection = client.collections.get("Yoruba_rag")


In [22]:
client.connect() 
print("Client reconnected.")
yoruba_collection = client.collections.get("Yoruba_rag")
first_object = all_nodes[0]
raw_properties = {
                "text": first_object.text,
                "domain": first_object.metadata.get('domain'),
                "source": first_object.metadata.get('source'),
                "url": first_object.metadata.get('url'),
                "chunks_no": first_object.metadata.get('chunks_no'),
                "vector" : embedder._get_text_embedding(first_object.text)
    
            }


data_properties = clean_properties(raw_properties)

# 👉 Correct method
yoruba_collection.data.insert(data_properties)

response = yoruba_collection.aggregate.over_all(total_count=True)
print(response.total_count)  # Should output 1

client.close()
print("Client connection closed.")


Client reconnected.
1
Client connection closed.


In [ ]:
from tqdm import tqdm

client.connect()
print("Client reconnected.")

yoruba_collection = client.collections.get("Yoruba_rag")

remaining_data = all_nodes[1:]  # your remaining nodes

print(f"Total items to insert: {len(remaining_data)}")

with yoruba_collection.batch.dynamic() as batch:
    for item in tqdm(remaining_data, desc="Inserting chunks"):
        
        # --- Prepare properties ---
        raw_properties = {
            "text": item.text,
            "domain": item.metadata.get("domain"),
            "source": item.metadata.get("source"),
            "url": item.metadata.get("url"),
            "chunks_no": item.metadata.get("chunks_no"),
        }

        # clean null values
        data_properties = clean_properties(raw_properties)

        # --- Compute AfriBERTa embedding ---
        vector = embedder._get_text_embedding(item.text)

        # --- Add to batch ---
        batch.add_object(
            properties=data_properties,
            vector=vector
        )

# ---- Check ----
response = yoruba_collection.aggregate.over_all(total_count=True)
print("Total count in Weaviate:", response.total_count)

client.close()
print("Client connection closed.")


Client reconnected.
Total items to insert: 402100


Inserting chunks:  41%|████      | 164985/402100 [27:40<39:24, 100.30it/s] 

In [9]:
import weaviate
from tqdm import tqdm
from typing import List, Dict
import math

# --- Your Cleaning Helpers ---
def clean_properties(props: dict) -> dict:
    """Replaces None/NaN values in dictionary properties with an empty string."""
    clean = {}
    for k, v in props.items():
        if v is None:
            clean[k] = ""
        elif isinstance(v, float) and math.isnan(v): 
            clean[k] = ""
        else:
            clean[k] = str(v) 
    return clean

# --- Assume client and collection are initialized and connected ---
# client = weaviate.connect_to_wcs(...) 
COLLECTION_NAME = "Yoruba_rag"
# nodes: List[TextNode] = ... 
BATCH_SIZE = 512 

print(f"\n--- Starting Cleaned Batch Upload of {len(all_nodes)} Nodes ---")

try:
    # 1. Ensure the client is connected (from the last fix)
    if not client.is_connected():
        print("Reconnecting Weaviate client...")
        client.connect()
        print("Client reconnected.")

    # 2. Get the collection handle
    collection = client.collections.get(COLLECTION_NAME)

    # 3. Start the batch operation
    with collection.batch.dynamic() as batch:
        
        for node in tqdm(all_nodes, desc="Cleaning, Batching, and Inserting Nodes"):
            
            raw_properties = {
                "content": node.text,
                "domain": node.metadata.get('domain'),
                "source": node.metadata.get('source'),
                "url": node.metadata.get('url'),
                "chunks_no": node.metadata.get('chunks_no'),
            }
            
            data_properties = clean_properties(raw_properties)
            
            # 4. CRITICAL FIX: Use the likely correct method name: add_object()
            batch.add_object(
                properties=data_properties 
                # collection is inherited from the context manager
            )
            
    print(f"\n✅ Dynamic Batch Upload Complete! All {len(all_nodes)} nodes processed.")
    
    # Verification
    count_result = collection.aggregate.over_all(total_count=True)
    print(f"✅ Database Count: {count_result.total_count} objects in collection.")

except Exception as e:
    print(f"❌ A critical error occurred during batch insertion: {e}")
finally:
    if 'client' in locals() and client.is_connected():
        client.close()
        


--- Starting Cleaned Batch Upload of 402101 Nodes ---
Reconnecting Weaviate client...
Client reconnected.


Cleaning, Batching, and Inserting Nodes: 100%|██████████| 402101/402101 [18:01<00:00, 371.86it/s] 



✅ Dynamic Batch Upload Complete! All 402101 nodes processed.
✅ Database Count: 0 objects in collection.
Client connection closed.


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import login

import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import login

from llama_index.core.base.embeddings.base import BaseEmbedding

from llama_index.core import Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import weaviate
from weaviate.classes.config import Property, DataType

In [ ]:
# ✅ LlamaIndex-AfriBERTa Embedding (Mean Pooling + Normalize)
from llama_index.core.embeddings import BaseEmbedding
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.preprocessing import normalize
from pydantic import PrivateAttr

class AfriBERTaEmbedding(BaseEmbedding):
    _model: AutoModel = PrivateAttr()
    _tokenizer: AutoTokenizer = PrivateAttr()
    _device: str = PrivateAttr()

    def __init__(self, model_name: str = "castorini/afriberta_base", **kwargs):
        super().__init__(**kwargs)
        self._device = "cuda" if torch.cuda.is_available() else "cpu"
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModel.from_pretrained(model_name).to(self._device)
        self._model.eval()

    @classmethod
    def class_name(cls) -> str:
        return "AfriBERTaEmbedding"

    def _mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        )
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def _embed(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        inputs = self._tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self._device)
        
        with torch.no_grad():
            outputs = self._model(**inputs)
            embeddings = self._mean_pooling(outputs.last_hidden_state, inputs["attention_mask"])
            embeddings = normalize(embeddings.cpu().numpy(), norm="l2")
        return embeddings.tolist()

    def _get_query_embedding(self, query: str):
        return self._embed(query)[0]

    def _get_text_embedding(self, text: str):
        return self._embed(text)[0]

    async def _aget_query_embedding(self, query: str):
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str):
        return self._get_text_embedding(text)

In [ ]:
# 2. Create LlamaIndex embedding model (NOT LangChain!)
embed_model = AfriBERTaEmbedding(
    model_name="castorini/afriberta_base"
)

In [ ]:

import os
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Force sync CUDA errors for debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model_name = "castorini/afriberta_base"

# Load tokenizer & model from SAME checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device).eval()

def mean_pooling(last_hidden_state, attention_mask):
    # average pooling of token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    return (last_hidden_state * input_mask_expanded).sum(1) / input_mask_expanded.sum(1).clamp(min=1e-9)

@torch.no_grad()
def embed_text(texts, max_length=512):
    """Return embeddings as numpy arrays"""
    encoded = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    outputs = model(**encoded)
    embeddings = mean_pooling(outputs.last_hidden_state, encoded["attention_mask"])
    return embeddings.cpu().numpy()

# 🔹 Test it
vecs = embed_text(["Ẹ káàrọ̀, bí o ṣe wa?", "Hello world!"])
print("Embeddings shape:", vecs.shape)


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
# =============================
embeddings_with_metadata = []
for node in nodes:
    text = node.get_content()
    emb = embed_model.get_text_embedding(text)

    item = {
        "embedding": emb,
        "metadata": {
            "text": text,
            "domain": node.metadata.get("domain", ""),
            "source": node.metadata.get("source", ""),
            "url": node.metadata.get("url", ""),
        },
    }
    embeddings_with_metadata.append(item)

print(f"✅ Created {len(embeddings_with_metadata)} embeddings")

In [ ]:
# 5. Clean helpers
# -------------------------------
def clean_vector(vec):
    arr = np.array(vec, dtype=np.float32).flatten()
    arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
    return arr.tolist()

def clean_properties(props: dict) -> dict:
    clean = {}
    for k, v in props.items():
        if v is None:
            clean[k] = ""
        elif isinstance(v, float) and math.isnan(v):
            clean[k] = ""
        else:
            clean[k] = str(v)
    return clean

# -------------------------------
# 6. Generate embeddings + batch insert
# -------------------------------
with yoruba_collection.batch.dynamic() as batch:
    for node in nodes:
        text = node.get_content(metadata_mode="all")  # node text
        metadata = node.metadata

        properties = clean_properties(metadata)
        properties["text"] = text

        # Generate embedding
        vector = clean_vector(embed_model.get_text_embedding(text))

        # Add to Weaviate
        batch.add_object(
            properties=properties,
            vector=vector,
        )

print("✅ Indexing complete (all nodes pushed to Weaviate Cloud)")

In [ ]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import Document, VectorStoreIndex, StorageContext
import weaviate

In [ ]:
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="YorubaChunk",
    text_key="text",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
